In [3]:
import pandas as pd
import os

# We set the display options to ensure we can see columns clearly later
pd.set_option('display.max_columns', None)

In [4]:
# Exact path provided by you
raw_data_path = '/Users/maneeshkoti/Documents/ecommerce-churn-prediction/data/raw/online_retail_II.xlsx'

# Verify the file exists before trying to load it
if os.path.exists(raw_data_path):
    print("Success: File path found.")
else:
    print(f"Error: File not found at {raw_data_path}")
    print("Please check for typos in the path.")

Success: File path found.


In [5]:
try:
    # Load the Excel file object (lighter than loading the whole dataframe)
    xls = pd.ExcelFile(raw_data_path, engine='openpyxl')
    
    print("File loaded successfully.")
    print(f"Sheets found in this file: {xls.sheet_names}")
    
except Exception as e:
    print(f"An error occurred while loading the file: {e}")

File loaded successfully.
Sheets found in this file: ['Year 2009-2010', 'Year 2010-2011']


In [6]:
# We define the standard sheet names for the Online Retail II dataset
# We use 'try-except' blocks in case your specific Excel file has different names
dfs = []

try:
    print("Attempting to load 'Year 2009-2010'...")
    df_09_10 = pd.read_excel(xls, sheet_name='Year 2009-2010', engine='openpyxl')
    dfs.append(df_09_10)
    print(f"Loaded 'Year 2009-2010': {df_09_10.shape[0]} rows")
except ValueError:
    print("Sheet 'Year 2009-2010' not found.")

try:
    print("Attempting to load 'Year 2010-2011'...")
    df_10_11 = pd.read_excel(xls, sheet_name='Year 2010-2011', engine='openpyxl')
    dfs.append(df_10_11)
    print(f"Loaded 'Year 2010-2011': {df_10_11.shape[0]} rows")
except ValueError:
    print("Sheet 'Year 2010-2011' not found.")

# If specific sheet names failed, try loading the first sheet by default
if not dfs:
    print("Specific sheet names not found. Loading the first sheet...")
    df_default = pd.read_excel(raw_data_path, sheet_name=0, engine='openpyxl')
    dfs.append(df_default)
    print(f"Loaded default sheet: {df_default.shape[0]} rows")

# Combine all loaded dataframes
df = pd.concat(dfs, ignore_index=True)
print("-" * 30)
print(f"Total Rows Loaded: {df.shape[0]}")
print(f"Total Columns: {df.shape[1]}")
print(f"Column Names: {list(df.columns)}")

Attempting to load 'Year 2009-2010'...
Loaded 'Year 2009-2010': 525461 rows
Attempting to load 'Year 2010-2011'...
Loaded 'Year 2010-2011': 541910 rows
------------------------------
Total Rows Loaded: 1067371
Total Columns: 8
Column Names: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


In [8]:
import json

# 1. Ensure InvoiceDate is datetime (critical for date range check)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# 2. Calculate Metrics required by the JSON schema
total_rows = len(df)
total_columns = len(df.columns)

# Missing values per column
missing_values = df.isnull().sum().to_dict()

# Count duplicates
duplicate_rows = df.duplicated().sum()

# Date Range
start_date = df['InvoiceDate'].min().strftime('%Y-%m-%d')
end_date = df['InvoiceDate'].max().strftime('%Y-%m-%d')

# Specific Data Quality Checks
negative_quantities = (df['Quantity'] < 0).sum()
cancelled_invoices = df['Invoice'].astype(str).str.startswith('C').sum()
missing_customer_ids = df['Customer ID'].isnull().sum()
missing_pct = (missing_customer_ids / total_rows) * 100

# 3. Construct the Dictionary matching the Schema
data_quality_summary = {
    "total_rows": int(total_rows),
    "total_columns": int(total_columns),
    "missing_values": {k: int(v) for k, v in missing_values.items()},
    "duplicate_rows": int(duplicate_rows),
    "date_range": {
        "start": start_date,
        "end": end_date
    },
    "negative_quantities": int(negative_quantities),
    "cancelled_invoices": int(cancelled_invoices),
    "missing_customer_ids": int(missing_customer_ids),
    "missing_customer_ids_percentage": float(round(missing_pct, 2))
}

# 4. Save to JSON file
output_path = '/Users/maneeshkoti/Documents/ecommerce-churn-prediction/data/raw/data_quality_summary.json'

with open(output_path, 'w') as f:
    json.dump(data_quality_summary, f, indent=4)

print(f"Success! Data quality summary saved to: {output_path}")
print("-" * 30)
print(json.dumps(data_quality_summary, indent=4))

Success! Data quality summary saved to: /Users/maneeshkoti/Documents/ecommerce-churn-prediction/data/raw/data_quality_summary.json
------------------------------
{
    "total_rows": 1067371,
    "total_columns": 8,
    "missing_values": {
        "Invoice": 0,
        "StockCode": 0,
        "Description": 4382,
        "Quantity": 0,
        "InvoiceDate": 0,
        "Price": 0,
        "Customer ID": 243007,
        "Country": 0
    },
    "duplicate_rows": 34335,
    "date_range": {
        "start": "2009-12-01",
        "end": "2011-12-09"
    },
    "negative_quantities": 22950,
    "cancelled_invoices": 19494,
    "missing_customer_ids": 243007,
    "missing_customer_ids_percentage": 22.77
}


In [9]:
# 1. Standardize Column Names (Map Excel names to Project Standard)
df.rename(columns={
    'Invoice': 'InvoiceNo',
    'Price': 'UnitPrice',
    'Customer ID': 'CustomerID'
}, inplace=True)

# 2. Save to CSV (This becomes the input for the next phase)
output_csv_path = '/Users/maneeshkoti/Documents/ecommerce-churn-prediction/data/raw/online_retail_raw.csv'
df.to_csv(output_csv_path, index=False)

print(f"Success! Standardized raw data saved to: {output_csv_path}")
print(f"Columns: {list(df.columns)}")

Success! Standardized raw data saved to: /Users/maneeshkoti/Documents/ecommerce-churn-prediction/data/raw/online_retail_raw.csv
Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
